In [6]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

url_telecomX_data = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json'
data = pd.read_json(url_telecomX_data)

In [7]:
# Normaliza as colunas aninhadas
df_customer = pd.json_normalize(data['customer'])
df_phone = pd.json_normalize(data['phone'])
df_internet = pd.json_normalize(data['internet'])
df_account = pd.json_normalize(data['account'])

# Junta tudo em um único DataFrame
df_final = pd.concat([
    data[['customerID', 'Churn']], # Pegando as colunas principais
    df_customer,
    df_phone,
    df_internet,
    df_account
], axis=1)

df_final.info()
df_final.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [8]:
# Converte Churn para 0 e 1
df_final['Churn'] = df_final['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Muda Changes.Total para numérico
df_final['Charges.Total'] = pd.to_numeric(df_final['Charges.Total'], errors='coerce')

# Preenche os valores nulos com a mediana da coluna
mediana = df_final['Charges.Total'].median()
df_final['Charges.Total'] = df_final['Charges.Total'].fillna(mediana)

# Criação da coluna Contas_Diarias como base no faturamento diário
df_final['Contas_Diarias'] = df_final['Charges.Monthly']/30

In [9]:
# análise descritiva das colunas numéricas: mean - média, std - desvio padrão, 50% - mediana
df_final.describe().T

# análise descritiva das colunas categóricas:
df_final.describe(include=['object']).T

,count,unique,top,freq
customerID,7267,7267,9995-HOTOH,1
gender,7267,2,Male,3675
Partner,7267,2,No,3749
Dependents,7267,2,No,5086
PhoneService,7267,2,Yes,6560
MultipleLines,7267,3,No,3495
InternetService,7267,3,Fiber optic,3198
OnlineSecurity,7267,3,No,3608
OnlineBackup,7267,3,No,3182
DeviceProtection,7267,3,No,3195


In [10]:
# Entendendo como a variável 'churn' (evasão) está distribuída entre os clientes
# Conta os valores de churn
churn_counts = df_final['Churn'].value_counts()

labels = ['Ficaram', 'Cancelaram'] # Rótulos

# Cria a figura do gráfico
fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=churn_counts.values,
    hole=.4,
    marker_colors=['#0099C6', '#DC3912'] # Azul e vermelho
)])
fig.update_layout(
    title_text='Proporção de Clientes (Churn)<br>Ficaram vs Cancelaram',
    title_x=0.5, # Centraliza o título
    height=500,
    width=550
)
fig.show()

In [12]:
# Mapeando os valores de Churn para rótulos mais claros para os gráficos
df_final['Churn_Label'] = df_final['Churn'].map({0: 'Ficou', 1: 'Cancelou'})

# Gráfico de Contagem de Churn por Tipo de Contrato
fig = px.histogram(df_final,
                   x='Contract',
                   color='Churn_Label',
                   barmode='group',
                   title='Contagem de Churn por Tipo de Contrato',
                   labels={'Contract': 'Tipo de Contrato', 'count': 'Número de Clientes'},
                   color_discrete_map={'Ficou': '#0099C6', 'Cancelou': '#DC3912'})

fig.show()

In [15]:
# Gráfico Percentual de Churn por Método de Pagamento
fig = px.histogram(df_final,
                   x='PaymentMethod',
                   color='Churn_Label',
                   barnorm='percent',
                   title='Proporção Percentual de Churn por Método de Pagamento',
                   labels={'PaymentMethod': 'Método de Pagamento', 'percent': 'Percentual de Clientes'},
                   color_discrete_map={'Ficou': '#0099C6', 'Cancelou': '#DC3912'})

# Melhora a legibilidade dos rótulos do eixo X
fig.update_xaxes(tickangle=45)

fig.show()

In [17]:
# Valores númericos mais associados à evasão

# Gráfico Box Plot para Tempo de Contrato (tenure) vs. Churn
fig = px.box(df_final,
             x='Churn_Label',
             y='tenure',
             color='Churn_Label', # Colore as caixas de acordo com o grupo
             title='Distribuição do Tempo de Contrato (Meses) por Status de Churn',
             labels={'Churn_Label': 'Status do Cliente', 'tenure': 'Tempo de Contrato (Meses)'},
             color_discrete_map={'Ficou': '#0099C6', 'Cancelou': '#DC3912'})
fig.show()


In [19]:
# Gráfico Box Plot para Taxa Mensal vs. Churn
fig = px.box(df_final,
             x='Churn_Label',
             y='Charges.Monthly',
             color='Churn_Label',
             title='Distribuição da Taxa Mensal por Status de Churn',
             labels={'Churn_Label': 'Status do Cliente', 'Charges.Monthly': 'Taxa Mensal (R$)'},
             color_discrete_map={'Ficou': '#0099C6', 'Cancelou': '#DC3912'})
fig.show()

In [20]:
# Histograma para Tempo de Contrato (tenure) vs. Churn
fig = px.histogram(df_final,
                   x='tenure',
                   color='Churn_Label',
                   title='Distribuição da Frequência do Tempo de Contrato por Churn',
                   labels={'tenure': 'Tempo de Contrato (Meses)', 'count': 'Número de Clientes'},
                   marginal='box', # Adiciona um box plot na margem para reforçar a análise
                   color_discrete_map={'Ficou': '#0099C6', 'Cancelou': '#DC3912'})
fig.show()

# Relatório

## 1. Introdução

Este relatório apresenta os resultados de uma análise exploratória de dados (EDA) sobre o churn de clientes na Telecom X. O principal objetivo deste estudo é identificar os fatores e os perfis de clientes que estão mais associados ao cancelamento de serviços. A empresa enfrenta um alto índice de evasão, e a compreensão desses padrões é o primeiro passo fundamental para que a equipe de Data Science possa desenvolver modelos preditivos e, consequentemente, para que a área de negócios possa criar estratégias de retenção mais eficazes e direcionadas.

## 2. Limpeza e Tratamento de Dados (ETL)

A base de dados foi fornecida em um arquivo JSON, o que exigiu um processo de Extração, Transformação e Carga (ETL) para adequá-la à análise. As seguintes etapas foram executadas:

    Normalização de Dados Aninhados: Inicialmente, os dados estavam em um formato JSON aninhado. Foi necessário aplicar uma normalização para transformar os dicionários internos em colunas individuais, criando uma estrutura tabular adequada para a análise.

    Correção de Tipos de Dados: A coluna Charges.Total (Cobrança Total), que deveria ser numérica, foi identificada como tipo object (texto). Isso ocorria porque valores vazios (' ') estavam presentes, impedindo a correta interpretação numérica.

    Tratamento de Valores Nulos: Após a conversão da coluna Charges.Total para o formato numérico, os valores textuais se tornaram nulos (NaN). Identificamos 11 registros nessas condições. Para manter a integridade do dataset, esses valores foram substituídos pela mediana da coluna, uma medida de tendência central robusta a outliers.

    Engenharia de Features: Como forma de enriquecer a análise, foi criada a coluna Contas_Diarias, calculada a partir da divisão da cobrança mensal (Charges.Monthly) por 30. Esta feature permite uma visão mais granular do faturamento.

## 3. Análise Exploratória de Dados (EDA)

A análise foi focada em cruzar as características dos clientes com a variável Churn para encontrar padrões.

### 3.1. Perfil Geral dos Clientes e Taxa de Churn

A análise descritiva revelou uma taxa de churn geral de 26,5% no dataset. O perfil do cliente médio da Telecom X é caracterizado por:

    Tempo de Contrato (Fidelidade): A média de permanência é de 32 meses, porém com alta variação. Metade dos clientes está na base há 29 meses ou menos.

    Faturamento: A cobrança mensal média é de R$ 64,76.

    Contrato e Pagamento: O tipo de contrato mais comum é o mensal (Month-to-month) e o método de pagamento mais utilizado é o cheque eletrônico (Electronic check).

    Serviços: O serviço de internet mais popular é o de Fibra Óptica.

### 3.2. Fatores Comportamentais e o Churn

Ao aprofundar a análise com visualizações, emergiram padrões claros:

    Tipo de Contrato vs. Churn: O tipo de contrato é o preditor mais forte de churn. Clientes com contrato mensal apresentam uma taxa de cancelamento alarmante de 41,3%, enquanto para clientes com contrato de 2 anos, a taxa é de apenas 2,8%. Isso indica que a falta de um compromisso de longo prazo é um fator de risco crítico.

    Método de Pagamento vs. Churn: Clientes que utilizam cheque eletrônico como forma de pagamento cancelam significativamente mais (43,8% de taxa de churn) em comparação com outros métodos como transferência, cartão de crédito ou boleto, cujas taxas de churn são muito próximas e inferiores a 19%.

### 3.3. Fatores Numéricos e o Churn

A análise das variáveis numéricas confirmou o perfil de risco:

    Ciclo de Vida do Cliente: Clientes que cancelam o serviço o fazem, majoritariamente, no início de sua jornada com a empresa. O box plot e o histograma mostram que a mediana de tempo de contrato para clientes que cancelaram é de apenas 10 meses, contra 40 meses para os que permaneceram. O pico de cancelamentos ocorre nos primeiros meses.

    Valor do Plano: Existe uma forte correlação entre o valor pago e a propensão a cancelar. A mediana da taxa mensal para clientes que cancelaram é de aproximadamente R$ 80, enquanto para os que ficaram é de R$ 65. Clientes com planos mais caros tendem a cancelar mais.

## 4. Conclusões e Insights

A análise de dados permitiu traçar um perfil claro e acionável do cliente com alto risco de churn na Telecom X. Este cliente tipicamente possui as seguintes características:

    Contrato: Possui um contrato flexível, do tipo mensal.

    Tempo na Base: É um cliente recente, com menos de 1 ano na empresa.

    Pagamento: Utiliza cheque eletrônico para pagar suas faturas.

    Valor: Paga uma taxa mensal elevada, geralmente acima de R$ 70.

    (Insight adicional a ser explorado): Frequentemente não possui serviços de valor agregado, como suporte técnico ou segurança online.

## 5. Recomendações Estratégicas

Com base nos insights gerados, recomendamos as seguintes ações para serem exploradas pelas equipes de negócio e marketing, a fim de reduzir a evasão de clientes:

    Foco na Retenção Inicial: Implementar um programa de onboarding e acompanhamento proativo para clientes nos primeiros 6 a 12 meses, que é o período mais crítico. O objetivo é garantir uma boa experiência inicial e demonstrar o valor do serviço.

    Incentivo a Contratos de Longo Prazo: Criar campanhas ativas para migrar clientes do plano mensal para contratos anuais ou bianuais, oferecendo benefícios claros como descontos na mensalidade, upgrade de serviço ou pacotes de canais/dados adicionais.

    Otimização dos Métodos de Pagamento: Investigar possíveis problemas de usabilidade ou atrito no pagamento com cheque eletrônico. Ao mesmo tempo, incentivar a adoção de métodos com menor churn, como débito automático em conta ou cartão de crédito, oferecendo pequenos descontos para a troca.

    Gestão de Valor para Planos "Premium": Para clientes com planos de alto valor, a empresa deve garantir que a percepção de custo-benefício seja alta. Ações como um "check-up" anual de plano, ofertas de serviços agregados (como TechSupport) ou acesso a conteúdo exclusivo podem aumentar a percepção de valor e a lealdade.